In [1]:
import sys
sys.path.insert(0, '../../src')
sys.path.append('../../src/d00_utils')
sys.path.append('../../src/d01_data')
sys.path.append('../../src/d02_processing')
sys.path.append('../../src/d03_modelling')
import extractions as extract
import transformations as transf
import training as train
import import_data as imp
import inference as inf
import time  
from dateutil.relativedelta import relativedelta
from datetime import date
import config as conf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAXResults

# Load Prices and Volumes

In [3]:
# SPAIN

# Prices
price = imp.get_prices_junta()
imp.load_prices_bbdd(price)
# Volumes
imp.load_volumes_bbdd(imp.get_volumes_ibo())

# US

crops_us = conf.crop_list_usa
for i in range(0, len(crops_us)):
    crop = crops_us[i][0]
    crop_abb = crops_us[i][1]
    # Prices
    price = imp.get_prices_usda(crop,crop_abb)
    imp.load_prices_bbdd(price)
    # Volumes
    imp.load_volumes_bbdd(imp.get_volumes_usda(crop,crop_abb))

6  new prices added
308  new volumes added
427  new prices added
1135  new volumes added
61  new prices added
770  new volumes added
256  new prices added
410  new volumes added


# Train models

In [ ]:
crop_list = conf.crop_list_arima
crop_list[:0]= (conf.crop_list_sarima)
crop_list[:0]= (conf.crop_list_sarimax)

for i in range(0, len(crop_list)):

    crop = crop_list[i][0]
    ctry = crop_list[i][1]
    trade_ctry = crop_list[i][2]
    ctgr = crop_list[i][3]
    mdel = crop_list[i][4]
    regn = crop_list[i][5]
    pkge = crop_list[i][6]
    crcy = crop_list[i][7]
    msre = crop_list[i][8]
    exog = crop_list[i][9]

    # Training

    train.train_arima_model(crop,ctry,trade_ctry,ctgr)

# Inference

In [ ]:
crop_list = conf.crop_list_arima
crop_list[:0]= (conf.crop_list_sarima)
crop_list[:0]= (conf.crop_list_sarimax)

for i in range(0, len(crop_list)):

    crop = crop_list[i][0]
    ctry = crop_list[i][1]
    trade_ctry = crop_list[i][2]
    ctgr = crop_list[i][3]
    mdel = crop_list[i][4]
    regn = crop_list[i][5]
    pkge = crop_list[i][6]
    crcy = crop_list[i][7]
    msre = crop_list[i][8]
    exog = crop_list[i][9]

    for k in range(2017, 2021):

        dt = date(k, 1, 1)
        start = str(dt + relativedelta(days=(7 - dt.weekday())))
        end = (date.today() + relativedelta(years=1)).strftime('%Y-12-31')
        
        if mdel == 'SARIMAX':
            mdel_vols = 'SARIMA'
            df_pred_vols = inf.get_prediction_vols(ctry,crop,trade_ctry,regn,ctgr,pkge,crcy,msre,mdel_vols,start,end)
            exog = df_pred_vols[df_pred_vols.Date_ref > date.today().strftime('%Y-01-01')].drop(columns=['Volume']).set_index('Date_ref')

        df_pred = inf.get_prediction(ctry,crop,trade_ctry,regn,ctgr,pkge,crcy,msre,mdel,exog,start,end)
        inf.load_predictions_db(df_pred,ctry,crop,trade_ctry,regn,ctgr,pkge,crcy,msre,mdel)